In [7]:
import os
import requests
import pandas as pd
import json
from dotenv import load_dotenv

In [59]:
load_dotenv()

True

In [60]:
covalent_API_KEY = os.getenv("Covalent-API-KEY")

## Get NFT Token IDS

In [61]:
url = "https://api.covalenthq.com/v1"
chain_id = "/1"

contract_address = "/0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB"
api_key = "/?key=" + covalent_API_KEY

url_tokens = url + chain_id + "/tokens" + contract_address + "/nft_token_ids" + api_key #+ "&format=csv"

nft_tokens = requests.get(url_tokens).json()

token_IDS = pd.DataFrame(nft_tokens['data']['items'])
token_IDS.head()

,contract_decimals,contract_name,contract_ticker_symbol,contract_address,supports_erc,logo_url,token_id
0,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,0
1,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,1
2,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,2
3,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,3
4,0,CRYPTOPUNKS,Ͼ,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,None,https://logos.covalenthq.com/tokens/0xb47e3cd8...,4


## Get NFT Contract Transactions

In [62]:
contract_address = "/0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB"
token_ID = "/200"

url_transactins = url + chain_id + "/tokens" + contract_address + "/nft_transactions" + token_ID + api_key
nft_transactions = requests.get(url_transactins).json()

transactions_df = pd.DataFrame(nft_transactions['data']['items'][0]['nft_transactions'][0])
transactions_df.head()

,block_signed_at,block_height,tx_hash,tx_offset,successful,from_address,from_address_label,to_address,to_address_label,value,value_quote,gas_offered,gas_spent,gas_price,fees_paid,gas_quote,gas_quote_rate,log_events
0,2022-03-17T03:51:37Z,14401717,0x5b0ee618e7eef1baefc510019edb2c9b5962ee1730e4...,6,True,0xc352b534e8b987e036a93539fd6897f53488e56a,None,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CRYPTOPUNKS (Ͼ),0,0.0,400000,48279,60000000000,2896740000000000,8.196203,2829.45752,"{'block_signed_at': '2022-03-17T03:51:37Z', 'b..."
1,2022-03-17T03:51:37Z,14401717,0x5b0ee618e7eef1baefc510019edb2c9b5962ee1730e4...,6,True,0xc352b534e8b987e036a93539fd6897f53488e56a,None,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CRYPTOPUNKS (Ͼ),0,0.0,400000,48279,60000000000,2896740000000000,8.196203,2829.45752,"{'block_signed_at': '2022-03-17T03:51:37Z', 'b..."


## GET Address Balances

In [63]:
address = "/msaki.eth"

url_bal = url + chain_id + "/address" + address + "/balances_v2" + api_key
balances = requests.get(url_bal).json()

balance_df = pd.DataFrame(balances['data']['items'])
balance_df.head()

,contract_decimals,contract_name,contract_ticker_symbol,contract_address,supports_erc,logo_url,last_transferred_at,type,balance,balance_24h,quote_rate,quote_rate_24h,quote,quote_24h,nft_data
0,18,Ether,ETH,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,None,https://www.covalenthq.com/static/images/icons...,None,cryptocurrency,11686119944272052348192,11101220213973664855979,2895.194300,2986.672900,3.383359e+07,3.315571e+07,None
1,18,ApeCoin,APE,0x4d224452801aced8b2f0aebe155379bb5d594381,[erc20],https://logos.covalenthq.com/tokens/1/0x4d2244...,2022-04-25T23:55:43Z,cryptocurrency,4387375327825867762092,4387375327825867762092,19.103985,19.389719,8.381635e+04,8.506998e+04,None
2,18,Bemil.io,BEM,0x3102931098e75111a6f57cdf9a3e68dd007e5dd1,[erc20],https://logos.covalenthq.com/tokens/1/0x310293...,2021-11-06T22:49:45Z,cryptocurrency,10238000000000000000000,10238000000000000000000,0.388418,NaN,3.976627e+03,NaN,None
3,6,USD Coin,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,[erc20],https://logos.covalenthq.com/tokens/1/0xa0b869...,2021-11-20T09:12:25Z,stablecoin,2545889125,2545889125,0.998554,1.056760,2.542209e+03,2.690393e+03,None
4,6,Tether USD,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,[erc20],https://logos.covalenthq.com/tokens/1/0xdac17f...,2022-04-23T15:21:23Z,stablecoin,1142525387,1142525387,1.000115,1.058784,1.142656e+03,1.209688e+03,None


## Get historical portfolio value over time

In [64]:
address = "/msaki.eth"

url_portfolio = url + chain_id + "/address" + address + "/portfolio_v2" + api_key
portfolio = requests.get(url_portfolio).json()

portfolio_df = pd.DataFrame(portfolio['data']['items'][0]['holdings'])

In [65]:
portfolio_df.head()

,timestamp,quote_rate,open,high,low,close
0,2022-04-27T00:00:00Z,2895.1943,"{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'..."
1,2022-04-26T00:00:00Z,2986.6729,"{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'..."
2,2022-04-25T00:00:00Z,2922.0510,"{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'..."
3,2022-04-24T00:00:00Z,2928.9316,"{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'..."
4,2022-04-23T00:00:00Z,2958.4500,"{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'...","{'balance': '11686125177139729763435', 'quote'..."


In [66]:
url_global_view = url + chain_id + "/nft_market" + api_key
global_view = requests.get(url_global_view).json()

global_view_df = pd.DataFrame(global_view['data']['items'])
global_view_df

,chain_id,collection_name,collection_address,volume_wei_24h,volume_quote_24h,avg_volume_wei_24h,avg_volume_quote_24h,contract_deployment_at,market_cap_wei,market_cap_quote,...,floor_price_wei_7d,floor_price_quote_7d,gas_quote_rate,quote_currency,opening_date,first_nft_image_token_id,first_nft_image,first_nft_image_256,first_nft_image_512,first_nft_image_1024
0,1,Rarible,0xd07dc4262bcdbf85190c01c996b4c06a461d2430,1000000000000000000,3.477595e+03,1000000000000000000,3477.5955,2020-05-27T11:49:56Z,6269271254011360000000000000000,2.180199e+16,...,137816704081529000,4.792708e+02,3477.5955,USD,2022-04-03,1,https://cloudflare-ipfs.com/ipfs/QmdU9h6dECWKR...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
1,1,Town Star,0xc36cf0cfcb5d905b8b513860db0cfe63f6cf9f5c,0,0.000000e+00,0,0.0000,2020-06-04T10:10:10Z,35996080896093600000000000,1.251798e+11,...,865000000000000000,3.008120e+03,3477.5955,USD,2022-04-03,None,None,None,None,None
2,1,Fantasy Islands - Sandbox,0x68aea268f92b715e073d2f5170039faa4b94a47b,0,0.000000e+00,0,0.0000,2021-08-20T00:00:55Z,1153800000000000000000000,4.012450e+09,...,7770666666666670000000,2.702324e+07,3477.5955,USD,2022-04-03,1,https://ipfs.io/ipfs/QmPqJyCtpQux4CZ4uVpJ9PdFv...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
3,1,BoredApeYachtClub,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,743000000000000000000,2.583854e+06,185750000000000000000,645963.4000,2021-04-22T03:03:16Z,311400386483699000000000,1.082925e+09,...,118649928843994000000,4.126165e+05,3477.5955,USD,2022-04-03,0,https://ipfs.io/ipfs/QmRRPWG96cmgTn2qSzjwr2qvf...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
4,1,Sandbox's LANDs,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,2390000000000000000,8.311453e+03,2390000000000000000,8311.4530,2019-12-04T07:51:55Z,263567420745161000000000,9.165809e+08,...,2330554166666670000,8.104725e+03,3477.5955,USD,2022-04-03,45,https://api.sandbox.game/lands/afc9cb8b-c281-4...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,Antonym,0x7e3ef31186d1bec0d3f35ad701d065743b84c790,5889000000000000000,2.047956e+04,309947368421053000,1077.8716,2022-03-01T13:40:37Z,13529453477274200000000,4.704996e+07,...,816532305865922000,2.839569e+03,3477.5955,USD,2022-04-03,0,https://ipfs.io/ipfs/QmaiGqsf79eXdttm8uxvj4zY6...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
96,1,ASMBrain,0xd0318da435dbce0b347cc6faa330b5a9889e3585,6380000000000000000,2.218706e+04,3190000000000000000,11093.5290,2021-12-02T08:09:18Z,13221381174274900000000,4.597862e+07,...,7487863415220100000,2.603976e+04,3477.5955,USD,2022-04-03,0,https://brains.alteredstatemachine.xyz/img/c5c...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
97,1,Sneaky Vampire Syndicate,0x219b8ab790decc32444a6600971c7c3718252539,1399000000000000000,4.865156e+03,466333333333333000,1621.7186,2021-09-06T23:35:08Z,13129656313206500000000,4.565963e+07,...,501976271186441000,1.745670e+03,3477.5955,USD,2022-04-03,1,https://ipfs.io/ipfs/bafybeibu3edpaeds5w2a23m6...,https://image-proxy.svc.prod.covalenthq.com/25...,https://image-proxy.svc.prod.covalenthq.com/51...,https://image-proxy.svc.prod.covalenthq.com/10...
98,1,Realms (for Adventurers),0x7afe30cb3e53dba6801aa0ea647a0ecea7cbe18d,64370000000000000000,2.238528e+05,5364166666666670000,18654.4020,2021-08-31T17:29:17Z,13113579031876000000000,4.560372e+07,...,23457

## Get Historical data for NFT collection

In [ ]:
collection_address = ""